# Generic COG Rendering in Lonboard

## Imports

In [1]:
import io

import numpy as np
from async_geotiff import GeoTIFF, Tile
from obstore.store import S3Store
from PIL import Image
from sidecar import Sidecar

from lonboard import Map, RasterLayer
from lonboard.raster import EncodedImage

## Create GeoTIFF object referencing remote S3 resource

In [2]:
BUCKET_URL = "https://s3.us-east-1.amazonaws.com/ds-deck.gl-raster-public"
COG_URL = "/cog/Annual_NLCD_LndCov_2024_CU_C1V1.tif"

Then we'll use Obstore and Async-GeoTIFF to open our remote file:

In [3]:
store = S3Store.from_url(BUCKET_URL, skip_signature=True)
geotiff = await GeoTIFF.open(COG_URL, store=store)

## Create Render Callback

Lonboard's COG support works by asynchronously fetching tiles _from Python_ and then transferring the tile to JavaScript for visualization.

In this initial version of our support, we require the user to create a "render callback" that transforms the COG input to a PNG-formatted RGB tile.

The benefit of this approach is that you can use _any Python code_ to perform the rendering characteristics you desire.

In [4]:
cmap_array = geotiff.colormap.as_array()


def render_paletted_tile(tile: Tile) -> EncodedImage:
    # data is (1, height, width), uint8 — single band with palette indices
    arr = tile.array.data[0]  # shape: (H, W)

    # Add alpha channel: fully transparent where nodata, opaque elsewhere
    alpha = np.full(arr.shape, 255, dtype=np.uint8)
    if tile.array.nodata is not None:
        alpha[arr == tile.array.nodata] = 0

    # Map palette indices to RGBA using fancy indexing
    rgba = np.empty((*arr.shape, 4), dtype=np.uint8)
    rgba[..., :3] = cmap_array[arr]  # (H, W, 3)
    rgba[..., 3] = alpha  # (H, W)

    # Serialize to PNG
    img = Image.fromarray(rgba, mode="RGBA")
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return EncodedImage(data=buf.getvalue(), mime_type="image/png")

In [5]:
layer = RasterLayer.from_async_geotiff(geotiff, render=render_paletted_tile)

In [6]:
m = Map(layer, height=1200)

In [7]:
sidecar = Sidecar()

In [8]:
with sidecar:
    display(m)